In [35]:
import os
import numpy as np
import pandas as pd
import astropy.coordinates as coord
from astroquery.ipac.ned import Ned
import yaml

os.chdir('/mnt/scratch/HCGs')
import hidefutils as utils

In [88]:
def median_separation(coos):
    # Calculate separations between all pairs of coordinates in coo_cores
    separations = []
    for i in range(len(coos)):
        for j in range(i + 1, len(coos)):
            separations.append(coos[i].separation(coos[j]).arcmin)

    # Compute the median separation
    median_separation = np.median(separations)
    return median_separation

def initial_velocity_dispersion(hcg, core_members):
    '''
    Calculate the velocity dispersion of a HCG from the systemic velocities of its members.
    Uses NED to get the systemic velocities.
    Parameters
    ----------
    g: HCG name
    '''
    with open('group_members.yml') as f:
        group_members = yaml.safe_load(f)
    with open('data_files.yml') as f:
        params = yaml.safe_load(f)
    df = pd.read_csv(f'params_tables/HIdef_hcg{hcg}.csv', na_values='--')
    # members = group_members[g]
    # cores = members['core']
    vcen = params[f'HCG {hcg}']['velocity']
    df = df.Name.str.replace('core: ', '')
    name_cores = core_members[hcg]
    vsys = np.zeros(len(name_cores))
    for i,name in enumerate(name_cores):
        try:
            ned_v = Ned.get_table(name, table='redshifts')
        except:
            vsys[i] = np.nan
        else:
            vsys[i] = np.ma.median(ned_v['Published Velocity'])
    delta_v_sq = np.array([(x-vcen)**2 for x in vsys])
    vdisp = np.sqrt(np.nanmean(delta_v_sq))
    delta_v_sq = np.array([(x-vcen)**2 for x in vsys])
    vdisp = np.sqrt(np.nanmean(delta_v_sq))
    return vdisp

def initial_center_of_mass(hcg, core_members):
    '''
    Calculates the center of mass of an HCG
    Params:
    ----------
    g: HCG ID
    '''
    with open('group_members.yml') as f:
        members = yaml.safe_load(f)
    with open('data_files.yml') as f:
        params = yaml.safe_load(f)

    # hcg_members = members[g]
    # core_members = hcg_members['core']
    name_cores = core_members[hcg]

    masses = pd.read_csv(f'/mnt/scratch/HCGs/stellar_masses_h{hcg}.csv', index_col='ID')
    core_masses = masses[masses.Name.isin(name_cores)]
    ra, dec, sm = core_masses.RA, core_masses.Dec, 10**core_masses.logMstar
    cen_ra, cen_dec = utils.center_radec(ra, dec, sm)
    return cen_ra, cen_dec

In [26]:
initial_core_members = {16: ['HCG 16A', 'HCG 16B', 'HCG 16C', 'HCG 16D', 'NGC 848'],
                        30: ['HCG 30A', 'HCG 30B', 'HCG 30C', 'HCG 30D'],
                        31: ['HCG 31A', 'HCG 31B', 'HCG 31C', 'HCG 31G', 'HCG 31Q'],
                        90: ['HCG 90A', 'HCG 90B', 'HCG 90C', 'HCG 90D'],
                        91: ['HCG 91A', 'HCG 91B', 'HCG 91C', 'HCG 91D'],
                        97: ['HCG 97A', 'HCG 97B', 'HCG 97C', 'HCG 97D', 'HCG 97E']}

In [127]:
hcg = 90
sigma = 3
t = pd.read_csv(f'/mnt/scratch/HCGs/params_tables/HIdef_hcg{hcg}.csv')
t.dropna(subset=['Name'], inplace=True)
ra, dec, sep_norm, rmag, vsys = t['RA'], t['Dec'], t['sep_norm'], t['rmag'], t['Vsys']
coos = coord.SkyCoord(ra, dec, unit='deg', frame='icrs')
t.Name = t.Name.str.replace('core: ', '')
m_cores = t.Name.isin(initial_core_members[hcg])
ra_cores = t[m_cores]['RA']
dec_cores = t[m_cores]['Dec']
coo_cores = coord.SkyCoord(ra_cores, dec_cores, unit='deg', frame='icrs')
medsep = median_separation(coo_cores)
vcore = t[t.Name == 'total']['Vsys'].values[0]
vdisp = initial_velocity_dispersion(hcg, initial_core_members)

# cenradec = Ned.query_object(f'HCG {hcg}')
# cencoo = coord.SkyCoord(cenradec[0]['RA'], cenradec[0]['DEC'], unit=('deg'), frame='icrs')
cenradec = initial_center_of_mass(hcg, initial_core_members)
cencoo = coord.SkyCoord(cenradec[0], cenradec[1], unit=('deg'), frame='icrs')
seps = cencoo.separation(coos).arcmin
# len(t), len(t[sep < 1])
min_rmag = np.min(rmag[m_cores])
# max_rmag = np.max(rmag[m_cores])
m = (seps < sigma*medsep) & (vsys > vcore - sigma*vdisp) & (vsys < vcore + sigma*vdisp) #& (rmag <= min_rmag + 3) 
# m = sep < 1
t[m]

,Unnamed: 0,HCG,Name,RA,Dec,Vsys,M_type,T_type,sep_kpc,sep_norm,...,HIflag,MHIlim_flag,gmag,e_gmag,rmag,e_rmag,Btot,logLB,logMs,e_logMs
0,0,90.0,2MASS J22012971-3157464,330.374136,-31.963035,2785.0,S0,-2.0,75.905,0.208,...,1.0,0.0,16.714,0.001,16.147,0.001,16.461,8.405,8.811,0.01
9,9,90.0,HCG 90A,330.507876,-31.869675,2603.0,Sa,1.0,39.835,0.109,...,1.0,0.0,12.719,0.000,11.904,0.000,12.563,9.964,10.811,0.01
10,10,90.0,HCG 90B,330.535137,-31.989869,2523.0,E0,-5.0,30.733,0.084,...,0.0,1.0,13.382,0.000,12.600,0.000,13.213,9.704,10.510,0.01
11,11,90.0,HCG 90C,330.513161,-31.973654,2512.0,E0,-5.0,22.065,0.060,...,0.0,1.0,13.696,0.000,12.910,0.000,13.528,9.578,10.389,0.01
12,12,90.0,HCG 90D,330.527009,-31.992715,2775.0,Im,10.0,32.075,0.088,...,0.0,1.0,13.361,0.000,12.577,0.000,13.192,9.712,10.574,0.01
13,13,90.0,ESO 466- G 046,330.683906,-31.991065,2318.0,S0a,0.0,83.018,0.227,...,0.0,1.0,14.767,0.000,14.041,0.000,14.576,9.159,9.830,0.01
14,14,90.0,ESO 466- G 044,330.566176,-31.756485,2818.0,S0,-2.0,105.790,0.289,...,0.0,1.0,15.183,0.000,14.524,0.000,14.966,9.003,9.587,0.01
15,15,90.0,total,330.526399,-31.937027,2635.0,NaN,NaN,0.000,0.000,...,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,11.230,NaN
17,17,90.0,ESO 466- G 047,330.691896,-31.955485,2556.0,Sbc,4.0,81.578,0.223,...,0.0,1.0,16.140,0.001,15.529,0.000,15.905,8.627,9.128,0.01


In [141]:
hcg = 97
sigma = 3
t = pd.read_csv(f'/mnt/scratch/HCGs/params_tables/HIdef_hcg{hcg}.csv')
t.dropna(subset=['Name'], inplace=True)
ra, dec, sep_norm, rmag, vsys = t['RA'], t['Dec'], t['sep_norm'], t['rmag'], t['Vsys']
coos = coord.SkyCoord(ra, dec, unit='deg', frame='icrs')
t.Name = t.Name.str.replace('core: ', '')
m_cores = t.Name.isin(initial_core_members[hcg])
gms = t[m_cores].logMs.values
gms.max() - gms.min()

1.0679999999999996

In [118]:
16.09263751/medsep

4.052608868563679

In [56]:
median_separation(coo_cores)

3.440824265663562